# Building a dataset to fine-tuning an embedding model

Creating a dataset to customize the embedding model (BAAI/bge-m3) for a specific domain (Legal) and language (Spanish).

Based on:

- https://www.philschmid.de/sagemaker-train-deploy-embedding-models
- https://github.com/virattt/financial-datasets/
- https://github.com/virattt/financial-datasets/blob/main/financial_datasets/prompts.py

In [1]:
import os
from groq import Groq
import json
import time

from datasets import Dataset

In [2]:
import sys
sys.path.append(os.path.abspath('../../'))

from src.etls.boe.scrapper import BOEScrapper

[2024-07-11 14:32:05,451] [16647] [INFO] [root] Initialized logging
[2024-07-11 14:32:05,453] [16647] [INFO] [root] Initializing logging
[2024-07-11 14:32:05,453] [16647] [INFO] [root] Initialized logging


# Config

In [3]:
QUESTIONS_PER_DOCUMENT = 100

In [4]:
OUTPUT_NAME_DATASET = "dariolopez/justicio-rag-embedding-qa"

In [5]:
URLS = list(set([
    # 
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2023-12203",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2015-10565",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2015-10566",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2015-11719",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-1985-5392",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2015-11719",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2013-12887",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2018-16673",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2006-21990",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2017-657",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2007-6115",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2004-21760",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-9347",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-21007",
    "https://www.boe.es/diario_boe/xml.php?id=BOJA-b-2017-90529",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2013-12632",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2000-544",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2015-11072",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-1978-31229",
    # 
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-1995-24292",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2008-2493",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2008-2492",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2014-7534",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-9347",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-1999-19448",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2013-12632",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-13605",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2023-12203",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2023-5366",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2022-11589",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2015-11724",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2011-15623",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-1986-10499",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2020-17264",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2023-16066",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2007-5825",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2022-14630",
    "https://www.boe.es/diario_boe/xml.php?id=BOJA-b-2024-90030",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2001-12716",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2009-5491",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2003-771",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2017-12902",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2003-20254",
    "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2004-4214"
]))

In [6]:
URLS.remove("https://www.boe.es/diario_boe/xml.php?id=BOE-A-2023-12203")
URLS.insert(0, "https://www.boe.es/diario_boe/xml.php?id=BOE-A-2023-12203")

In [7]:
URLS[0]

'https://www.boe.es/diario_boe/xml.php?id=BOE-A-2023-12203'

In [8]:
len(URLS)

40

In [9]:
SYSTEM_PROMPT = """
Como un experto en derecho y leyes españolas, tu tarea es crear preguntas y respuestas sobre el Boletín Oficial del Estado (BOE) de España.

Tu tarea consiste en crear pares de preguntas y respuestas independientes, sin referencia a ningún documento concreto. 

Estas preguntas y respuestas se utilizarán de forma independiente en futuras aplicaciones como la evaluación y el ajuste de LLM, en las que no se dispondrá de ningún documento de referencia.

Sigue las siguientes reglas:

1. Derivación directa: Las respuestas deben derivarse directamente del contenido proporcionado.
2. Preguntas autónomas: Asegúrate de que las preguntas se pueden responder completamente a partir de la información proporcionada y no implican la existencia de un documento más amplio.
3. Claridad y precisión: Las preguntas deben ser claras, precisas y no ambiguas.
4. Referencias prohibidas: Evita explícitamente frases como "según el documento", “según el capítulo”, "en el texto", "como se menciona en el artículo", o cualquier implicación de textos externos. No construyas preguntas que requieran conocer la estructura del documento o la ubicación de la información en el mismo.
5. Inclusión del contexto: Incluya la información específica del contenido que apoya la respuesta. El contexto debe permitir que la respuesta sea independiente de cualquier texto externo.
6. Suficiencia de la información: Si el contenido carece de información suficiente para formar una pareja completa de pregunta-respuesta, no fuerces una.
7. Respuestas originales: Crea las respuestas con tus propias palabras; no se permite la copia directa del contenido.
8. Asegúrate de responder siempre en español.
9. El contexto debería tener una longitud aproximada de 1000 caracteres. 

Salida:

La salida tendrá el siguiente formato JSON:

```
[
    {
        "question": "texto para la pregunta",
        “context”: "texto para el contexto",
        "answer": "texto para la respuesta"
    },
    {
        "question": "texto para la pregunta",
        “context”: "texto para el contexto",
        "answer": "texto para la respuesta"
    }
]
```

En la salida NO incluyas ningún texto extra, CONTESTA exclusivamente en formato JSON.

Ejemplo de salida bien generada:

```
[
    {
        "question": "¿Cuál es el papel de la Unión Europea en la política de vivienda?",
        "context": "El artículo 19 del Pilar Europeo de derechos sociales, incorpora la vivienda entre los principios y los derechos esenciales para el funcionamiento de los sistemas de bienestar europeo y, por último, la Carta de los Derechos Fundamentales de la Unión Europea aprobada por el Parlamento, el Consejo y la Comisión Europea el 7 de diciembre de 2000 establece en su artículo 34.3 que «con el fin de combatir la exclusión social y la pobreza, la Unión reconoce y respeta el derecho a una ayuda social y a una ayuda de vivienda para garantizar una existencia digna a todos aquellos que no dispongan de recursos suficientes».",
        "answer": "La Unión Europea ha avanzado en el reconocimiento del derecho a la vivienda de toda persona, y ha establecido principios y derechos esenciales para el funcionamiento de los sistemas de bienestar europeo."
    },
    {
        "question": "¿Qué sucede si el acreedor hipotecario es un gran tenedor de vivienda?",
        "context": "En el caso de que el acreedor hipotecario sea un gran tenedor de vivienda, el inmueble objeto de demanda sea la vivienda habitual del deudor hipotecario y se tenga constancia, conforme a los apartados anteriores, que éste se encuentra en situación de vulnerabilidad económica, no se admitirán las demandas de ejecución hipotecaria en las que no se acredite que la parte actora se ha sometido al procedimiento de conciliación o intermediación que a tal efecto establezcan las Administraciones Públicas competentes, en base al análisis de las circunstancias de ambas partes y de las posibles ayudas y subvenciones existentes conforme a la legislación y normativa autonómica en materia de vivienda.",
        "answer": "No se admitirán las demandas de ejecución hipotecaria si no se acredita que la parte actora se ha sometido al procedimiento de conciliación o intermediación."
    },
    {
        "question": "¿Cuál es el requisito para acreditar la concurrencia o no de vulnerabilidad económica de la parte ejecutada?",
        "context": "Para acreditar la concurrencia o no de vulnerabilidad económica de la parte ejecutada se deberá aportar documento acreditativo, de vigencia no superior a tres meses, emitido, previo consentimiento de éste, por los servicios de las Administraciones autonómicas y locales competentes en materia de vivienda, asistencia social, evaluación e información de situaciones de necesidad social y atención inmediata a personas en situación o riesgo de exclusión social que hayan sido específicamente designados conforme a la legislación y normativa autonómica en materia de vivienda.",
        "answer": "Un documento acreditativo emitido por los servicios competentes, con una vigencia no superior a tres meses."
    }
]
```

Ejemplo de salida mal generada:
```
Aquí te dejo 9 tripletas de pregunta/respuesta/contexto sobre el bloque de texto proporcionado:
[
    {
        "Pregunta": "...",
        "Respuesta": "",
        "Contexto": "..."
    },
    {
        'question': '¿Qué tipo de viviendas se promueven en zonas de mercado residencial tensionado?',
        'context': 'Artículo 17. Vivienda asequible incentivada.',
        'answer': 'Viviendas asequibles incentivadas.'
    },
    {
        "question": "¿Cuál es el derecho constitucional que se reconoce en el artículo 47 de la Constitución Española?",
        "context": "La Constitución española (CE) reconoce, en su artículo 47, el derecho al disfrute de una vivienda digna y adecuada e impone seguidamente a los poderes públicos el deber de promover las condiciones necesarias que garanticen la igualdad en el ejercicio de los derechos y el cumplimiento de los deberes constitucionales.",
        "answer": "El derecho al disfrute de una vivienda digna y adecuada."
    },
]
```

NUNCA menciones las palabras "documento", "texto", "presentación", "archivo", "tabla", “artículo”, "ley", “capítulo”, “preámbulo”, “título preliminar”, “disposición” o “disposiciones generales” en sus preguntas o respuestas.

Asegúrate SIEMPRE de que todas las preguntas y respuestas sean precisas, autónomas y pertinentes, sin basarse en ningún documento o texto original ni insinuar su existencia, evitando estrictamente cualquier invención o especulación.

Asegúrate SIEMPRE de que la respuesta esté contenida en el contexto.
"""

In [11]:
client = Groq(
    api_key=os.environ.get('GROQ_API_KEY'),
)

In [12]:
# groq limits: https://console.groq.com/settings/limits

def split_text_into_parts(text, part_size):
    parts = []
    start = 0
    while start < len(text):
        end = start + part_size
        parts.append(text[start:end])
        start = end
    return parts

In [13]:
data = []
errors = []
for url in URLS[::-1]:

    document_id = url.split("?id=")[1]
    
    boe_scrapper = BOEScrapper()
    # text = boe_scrapper.download_eli_document(url)
    text = boe_scrapper.download_document(url, output_text=True)

    # Split original text because of the Groq API limits: https://console.groq.com/settings/limits
    splitted = split_text_into_parts(text, part_size=15_500)
    print(f"The document {url} was splitted on {len(splitted)} texts")
    for text_splitted in splitted:
        time.sleep(66)  # groq limits: https://console.groq.com/settings/limits

        questions = QUESTIONS_PER_DOCUMENT // len(splitted)
        limit_questions = 16
        if questions > limit_questions:  # groq limits generation
            questions = limit_questions
        elif questions == 0:
            questions = 1
        print(f"Generating {questions} questions")

        # Generate questions
        query = f"Genera {questions} tripletas de pregunta/respuesta/contexto para el siguiente bloque de texto: {text_splitted}. Retorna en formato JSON."
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": SYSTEM_PROMPT,
                },
                {
                    "role": "user",
                    "content": query,
                },
            ],
            model="llama3-70b-8192",
            temperature=0,
            stream=False,
            # max_tokens=1024
        )

        try:
            content = chat_completion.choices[0].message.content

            json_start = content.find('[')
            cleaned_content = content[json_start:].replace('\n', '').replace('\r', '').replace(',]', ']').replace('```', '')

            if cleaned_content.strip():
                responses = json.loads(cleaned_content)
                for response in responses:
                    if len(response['context']) > 120:
                        row = {}
                        row['document'] = document_id
                        row['question'] = response['question']
                        row['context'] = response['context']
                        row['answer'] = response['answer']
                        data.append(response)
            
        except Exception as e:
            print(f"Se ha producido una excepción: {e}")
            print(f"Wrong generated on url {url}")
            print(chat_completion.choices[0].message.content)

[2024-07-11 14:32:05,614] [16647] [INFO] [download_document] Scrapping document: https://www.boe.es/diario_boe/xml.php?id=BOE-A-2003-771
/home/vant/justicio/env3.10/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
[2024-07-11 14:32:05,906] [16647] [INFO] [download_eli_document] Scrapping consolidated text (eli) from document: https://www.boe.es/eli/es-an/l/2002/12/16/5
[2024-07-11 14:32:06,478] [16647] [INFO] [download_eli_document] Scrapped consolidated text (eli) successfully from document: https://www.boe.es/eli/es-an/l/2002/12/16/5
[2024-07

The document https://www.boe.es/diario_boe/xml.php?id=BOE-A-2003-771 was splitted on 2 texts
Generating 16 questions
Generating 16 questions


[2024-07-11 14:34:35,175] [16647] [INFO] [download_document] Scrapping document: https://www.boe.es/diario_boe/xml.php?id=BOE-A-2020-17264
/home/vant/justicio/env3.10/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
[2024-07-11 14:34:35,780] [16647] [INFO] [download_eli_document] Scrapping consolidated text (eli) from document: https://www.boe.es/eli/es/lo/2020/12/29/3
[2024-07-11 14:34:36,693] [16647] [INFO] [download_eli_document] Scrapped consolidated text (eli) successfully from document: https://www.boe.es/eli/es/lo/2020/12/29/3
[2024-07-1

The document https://www.boe.es/diario_boe/xml.php?id=BOE-A-2020-17264 was splitted on 20 texts
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions
Generating 5 questions


[2024-07-11 14:58:07,806] [16647] [INFO] [download_document] Scrapping document: https://www.boe.es/diario_boe/xml.php?id=BOE-A-2015-11724
[2024-07-11 14:58:09,012] [16647] [INFO] [download_eli_document] Scrapping consolidated text (eli) from document: https://www.boe.es/eli/es/rdlg/2015/10/30/8
[2024-07-11 14:58:10,387] [16647] [INFO] [download_eli_document] Scrapped consolidated text (eli) successfully from document: https://www.boe.es/eli/es/rdlg/2015/10/30/8
[2024-07-11 14:58:10,389] [16647] [INFO] [download_document] Scrapped document successfully https://www.boe.es/eli/es/rdlg/2015/10/30/8


...

The document https://www.boe.es/diario_boe/xml.php?id=BOE-A-2018-16673 was splitted on 16 texts
Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions


[2024-07-12 01:41:26,660] [16647] [INFO] [groq._base_client] Retrying request to /openai/v1/chat/completions in 0.976010 seconds


Generating 6 questions
Generating 6 questions
Generating 6 questions
Generating 6 questions


[2024-07-12 01:46:24,600] [16647] [INFO] [download_document] Scrapping document: https://www.boe.es/diario_boe/xml.php?id=BOE-A-2022-14630
[2024-07-12 01:46:34,657] [16647] [WARNING] [urllib3.connectionpool] Retrying (Retry(total=4, connect=4, read=5, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f1d1df55900>, 'Connection to www.boe.es timed out. (connect timeout=10)')': /diario_boe/xml.php?id=BOE-A-2022-14630
[2024-07-12 01:46:46,675] [16647] [WARNING] [urllib3.connectionpool] Retrying (Retry(total=3, connect=3, read=5, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f1d1d4960e0>, 'Connection to www.boe.es timed out. (connect timeout=10)')': /diario_boe/xml.php?id=BOE-A-2022-14630
[2024-07-12 01:47:00,695] [16647] [WARNING] [urllib3.connectionpool] Retrying (Retry(total=2, connect=2, read=5, redirect=None, status=None)) after con

ConnectTimeout: HTTPSConnectionPool(host='www.boe.es', port=443): Max retries exceeded with url: /diario_boe/xml.php?id=BOE-A-2022-14630 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f1d1df553c0>, 'Connection to www.boe.es timed out. (connect timeout=10)'))

In [14]:
len(data)

2460

In [15]:
# Calculating the mean of the contexts length
contexts = []
for d in data:
    contexts.append(len(d['context']))
print(sum(contexts) / len(contexts))

301.3670731707317


In [16]:
# Obtener las posiciones de los 10 mayores elementos
posiciones = sorted(range(len(contexts)), key=lambda i: contexts[i], reverse=True)[:10]

# Imprimir las posiciones y los valores correspondientes
for pos in posiciones:
    print(f"Posición: {pos}, Valor: {contexts[pos]}")

Posición: 957, Valor: 1138
Posición: 1672, Valor: 1050
Posición: 1921, Valor: 1041
Posición: 1143, Valor: 973
Posición: 211, Valor: 937
Posición: 1670, Valor: 910
Posición: 953, Valor: 902
Posición: 125, Valor: 892
Posición: 1798, Valor: 892
Posición: 1823, Valor: 881


In [17]:
# Obtener las posiciones de los 10 mayores elementos
posiciones = sorted(range(len(contexts)), key=lambda i: contexts[i], reverse=False)[:10]

# Imprimir las posiciones y los valores correspondientes
for pos in posiciones:
    print(f"Posición: {pos}, Valor: {contexts[pos]}")

Posición: 977, Valor: 121
Posición: 1181, Valor: 121
Posición: 1276, Valor: 121
Posición: 1300, Valor: 121
Posición: 1597, Valor: 121
Posición: 263, Valor: 122
Posición: 279, Valor: 122
Posición: 757, Valor: 122
Posición: 951, Valor: 122
Posición: 1211, Valor: 122


In [18]:
data[415]

{'question': '¿Qué tipo de Entidades locales pueden crear las Comunidades Autónomas?',
 'context': 'Las Comunidades Autónomas, de acuerdo con lo dispuesto en sus respectivos Estatutos, podrán crear en su territorio comarcas u otras Entidades que agrupen varios Municipios, cuyas características determinen intereses comunes precisados de una gestión propia o demanden la prestación de servicios de dicho ámbito.',
 'answer': 'Comarcas u otras Entidades que agrupen varios Municipios.'}

In [19]:
# Convertir la lista de diccionarios a un diccionario de listas
data_dict = {key: [dic[key] for dic in data] for key in data[0]}

In [20]:
dataset = Dataset.from_dict(data_dict)

In [21]:
dataset

Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 2460
})

In [22]:
import huggingface_hub

huggingface_hub.login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/vant/.cache/huggingface/token
Login successful


In [23]:
dataset.push_to_hub(OUTPUT_NAME_DATASET + '-tmp-2')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dariolopez/justicio-rag-embedding-qa-tmp/commit/cb68b8bf970cb46f335c393d52c06c62c9a215de', commit_message='Upload dataset', commit_description='', oid='cb68b8bf970cb46f335c393d52c06c62c9a215de', pr_url=None, pr_revision=None, pr_num=None)